<a href="https://colab.research.google.com/github/ShovalBenjer/deep_learning_neural_networks/blob/main/Deep_exc_1_adir_shoval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import torch
from torch import nn

class NeuralNetXOR(nn.Module):
    def __init__(self, k, bypass=False, T=0.001):
        super(NeuralNetXOR, self).__init__()
        self.bypass = bypass
        self.hidden = nn.Linear(2, k)
        self.output = nn.Linear(k, 1)
        self.btu = lambda x: 1 / (1 + torch.exp(-x / T))
        self.bypass_layer = nn.Linear(2, 1) if bypass else None

    def weights_set(self, layer_name, w, b):
        layer = getattr(self, layer_name)
        if w.shape != layer.weight.data.shape or b.shape != layer.bias.data.shape:
            raise ValueError("Input weight or bias shapes do not match layer dimensions.")
        with torch.no_grad():
            layer.weight = nn.Parameter(w)
            layer.bias = nn.Parameter(b)

    def forward(self, x):
        hidden_out = self.btu(self.hidden(x))
        output_out = self.btu(self.output(hidden_out))
        if self.bypass:
            bypass_out = self.btu(self.bypass_layer(x))
            return output_out + bypass_out
        return output_out

def loss_fn(x, y, model):
    y_pred = model(x)
    return torch.sum((y - y_pred) ** 2)

x = torch.FloatTensor([[0., 0.], [0., 1.], [1., 0.], [1., 1.]])
y = torch.FloatTensor([[0.], [1.], [1.], [0.]])

model = NeuralNetXOR(k=2, bypass=False, T=0.001)

model.weights_set("hidden", torch.tensor([[20., -20.], [-20., 20.]]), torch.tensor([-10., -10.]))
model.weights_set("output", torch.tensor([[20., 20.]]), torch.tensor([-30.]))

if model.bypass:
    model.weights_set("bypass_layer", torch.tensor([[0., 0.]]), torch.tensor([0.]))

hidden_weights = model.hidden.weight.data
hidden_bias = model.hidden.bias.data
output_weights = model.output.weight.data
output_bias = model.output.bias.data

print("Hidden Layer Weights:", hidden_weights)
print("Hidden Layer Bias:", hidden_bias)
print("Output Layer Weights:", output_weights)
print("Output Layer Bias:", output_bias)

y_pred = model(x)
loss = loss_fn(x, y, model)
print("Loss:", loss.item())
print("Truth Table:")
for i in range(len(x)):
    print(f"Input: {x[i].tolist()}, Predicted Output: {y_pred[i].item()}, Expected Output: {y[i].item()}")


Hidden Layer Weights: tensor([[ 20., -20.],
        [-20.,  20.]])
Hidden Layer Bias: tensor([-10., -10.])
Output Layer Weights: tensor([[20., 20.]])
Output Layer Bias: tensor([-30.])
Loss: 2.0
Truth Table:
Input: [0.0, 0.0], Predicted Output: 0.0, Expected Output: 0.0
Input: [0.0, 1.0], Predicted Output: 0.0, Expected Output: 1.0
Input: [1.0, 0.0], Predicted Output: 0.0, Expected Output: 1.0
Input: [1.0, 1.0], Predicted Output: 0.0, Expected Output: 0.0
